In [73]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [74]:
og_survey_df = pd.read_csv('comma-survey.csv', index_col='RespondentID')
new_survey_df = pd.read_csv('new_comma_survey.csv', index_col='RespondentID')
survey_df = pd.concat([og_survey_df, new_survey_df])

In [75]:
question_cols = list(survey_df.columns)[:7]
demographic_cols = list(survey_df.columns)[7:11]

In [76]:
question_cols

['In your opinion, which sentence is more gramatically correct?',
 'Prior to reading about it above, had you heard of the serial (or Oxford) comma?',
 'How much, if at all, do you care about the use (or lack thereof) of the serial (or Oxford) comma in grammar?',
 'How would you write the following sentence?',
 'When faced with using the word "data", have you ever spent time considering if the word was a singular or plural noun?',
 'How much, if at all, do you care about the debate over the use of the word "data" as a singluar or plural noun?',
 'In your opinion, how important or unimportant is proper use of grammar?']

In [94]:
# Encode demographic responses

demographics = survey_df[demographic_cols].to_numpy()
# Define the ordinal features pipeline
enc = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', add_indicator=True)),
    ('encoder', OrdinalEncoder()),
    ('scaler', StandardScaler())
])
enc.fit(demographics)
X_demographics = enc.transform(demographics)

In [95]:
np.argwhere(np.isnan(X_demographics))

array([], shape=(0, 2), dtype=int64)

In [96]:
# Fit label encoders
labelEncoders = []
for q in question_cols:
    y_vals = survey_df[q].to_numpy()
    le = preprocessing.LabelEncoder()
    le.fit(y_vals)
    labelEncoders.append(le)
    

In [97]:
# Fit models
lr = LogisticRegression(multi_class='multinomial')
models = []
for idx, q in enumerate(question_cols):
    y_vals = survey_df[q].to_numpy()
    Y_vals = labelEncoders[idx].transform(y_vals)
    model = lr.fit(X_demographics, Y_vals)
    models.append(model)

/Users/evabatelaan/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [98]:
models

[LogisticRegression(multi_class='multinomial'),
 LogisticRegression(multi_class='multinomial'),
 LogisticRegression(multi_class='multinomial'),
 LogisticRegression(multi_class='multinomial'),
 LogisticRegression(multi_class='multinomial'),
 LogisticRegression(multi_class='multinomial'),
 LogisticRegression(multi_class='multinomial')]

In [99]:
rev_X = enc.inverse_transform(X_demographics)

ValueError: could not convert string to float: 'Male'

AttributeError: 'Pipeline' object has no attribute 'categories_'